<img src="http://drive.google.com/uc?export=view&id=1tpOCamr9aWz817atPnyXus8w5gJ3mIts" width=500px>

Proprietary content. © Great Learning. All Rights Reserved. Unauthorized use or distribution prohibited.

**PROJECT DESCRIPTION:** Classification is probably the most popular task that you would deal with in real life. Text in the form of blogs, posts, articles, 
etc. is written every second. It is a challenge to predict the information about the writer without knowing about him/her. We are going to 
create a classifier that predicts multiple features ofthe author of a given text. We have designed it as a Multi label classification problem.

**• DATASET:** Over 600,000 posts from more than 19 thousand bloggers The Blog Authorship Corpus consists of the collected 
posts of 19,320 bloggers gathered from blogger.com in August 2004. The corpus incorporates a total of 681,288 posts and over 140 million 
words - or approximately 35 posts and 7250 words per person. 

Each blog is presented as a separate file, the name of which indicates a 
blogger id# and the blogger’s self-provided gender, age, industry, and astrological sign. (All are labelled for gender and age butfor many, 
industry and/or sign is marked as unknown.) All bloggers included in the corpus fall into one of three age groups:
• 8240 "10s" blogs (ages 13-17),
• 8086 "20s" blogs(ages 23-27) and
• 2994 "30s" blogs (ages 33-47)

For each age group, there is an equal number of male and female bloggers.

Each blog in the corpus includes atleast 200 occurrences of common English words. All formatting has been stripped with two exceptions. 
Individual posts within a single blogger are separated by the date of the following post and links within a post are denoted by the label url 
link. 

Link to data set: https: //ww w.kaggle.com/rtatman/blog-authorship-corpus

• **PROJECT OBJECTIVE:**The need is to build a NLP classifier which can use input text parameters to determine the label/s of of the blog.
[ Total Score: 40 points]


1. Load the dataset (5 points)

  a. Tip: As the dataset is large, use fewer rows. Check what is working well on your
machine and decide accordingly.

Steps to import file directly from Kaggle

In [1]:
!pip install kaggle --quiet

In [2]:
#Make a directory for Kaggle
!mkdir .kaggle

mkdir: cannot create directory ‘.kaggle’: File exists


In [3]:
from google.colab import drive
drive.mount('/content/drive/',force_remount=True)

Mounted at /content/drive/


In [4]:
#Copy kaggle.json file. Change gdrive folder based on where you have saved your json file from Kaggle
!cp '/content/drive/MyDrive/AIML/kaggle.json' /content/.kaggle/kaggle.json

In [5]:
#Check if json file is there
!ls -l /content/.kaggle

total 4
-rw------- 1 root root 73 Apr 10 13:58 kaggle.json


In [6]:
!mkdir ~/.kaggle
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
!kaggle config set -n path -v{/content}
!chmod 600 /root/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists
- path is now set to: {/content}


Verify Kaggle connection

In [7]:
!kaggle datasets list

ref                                                         title                                              size  lastUpdated          downloadCount  
----------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
gpreda/reddit-vaccine-myths                                 Reddit Vaccine Myths                              223KB  2021-04-10 08:12:55           2369  
crowww/a-large-scale-fish-dataset                           A Large Scale Fish Dataset                          3GB  2021-02-17 16:10:44           1373  
dhruvildave/wikibooks-dataset                               Wikibooks Dataset                                   1GB  2021-02-18 10:08:27           1040  
promptcloud/careerbuilder-job-listing-2020                  Careerbuilder Job Listing 2020                     42MB  2021-03-05 06:59:52            228  
imsparsh/musicnet-dataset                                   MusicNet Dataset

In [8]:
!kaggle datasets download -d rtatman/blog-authorship-corpus

blog-authorship-corpus.zip: Skipping, found more recently modified local copy (use --force to force download)


In [10]:
!unzip -q '/content/{/content}/datasets/rtatman/blog-authorship-corpus/blog-authorship-corpus.zip'

replace blogtext.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [9]:
import pandas as pd

In [12]:
blog_data = pd.read_csv('/content/blogtext.csv',nrows=10000,index_col=False)

In [13]:
blog_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      10000 non-null  int64 
 1   gender  10000 non-null  object
 2   age     10000 non-null  int64 
 3   topic   10000 non-null  object
 4   sign    10000 non-null  object
 5   date    10000 non-null  object
 6   text    10000 non-null  object
dtypes: int64(2), object(5)
memory usage: 547.0+ KB


2. Preprocess rows of the “text” column (7.5 points)

  a. Remove unwanted characters

  b. Convert text to lowercase

  c. Remove unwanted spaces

  d. Remove stopwords



Remove id and date columns as those would not be useful to us

In [14]:
blog_data.drop(labels=['id','date'], axis=1,inplace=True)

Randomise the rows

In [15]:
blog_data=blog_data.sample(frac=1).reset_index(drop=True)

Data cleansing by removing unwanted characters, spaces, stop words etc. 

In [16]:
blog_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   gender  10000 non-null  object
 1   age     10000 non-null  int64 
 2   topic   10000 non-null  object
 3   sign    10000 non-null  object
 4   text    10000 non-null  object
dtypes: int64(1), object(4)
memory usage: 390.8+ KB


In [17]:
blog_data['age']=blog_data['age'].astype('object')

In [18]:
blog_data.head()

,gender,age,topic,sign,text
0,male,35,Technology,Aries,good ole texas http://www.nytimes.c...
1,male,35,Technology,Aries,Popular uprising in Basra! Coaliti...
2,male,35,Technology,Aries,I cannot believe the Supreme Court ...
3,female,24,indUnk,Aries,'Live In Tokyo / 5 Years In A LI...
4,male,23,Internet,Aquarius,Please note the following when...


Remove junks

In [19]:
import re

In [20]:
blog_data['cleansed_text']=blog_data['text'].apply(lambda x: re.sub(r'[^A-Za-z]+',' ',x))



```
# This is formatted as code
```

Convert text to lowercase.

In [21]:
blog_data['cleansed_text']=blog_data['cleansed_text'].apply(lambda x: x.lower())

Remove unwanted spaces

In [22]:
blog_data['cleansed_text']=blog_data['cleansed_text'].apply(lambda x: x.strip())

check a random record to see if the cleansing is applied properly

In [23]:
print("Data before cleansing : {}".format(blog_data['text'][1]))

Data before cleansing :            Popular uprising in Basra!  Coalition forces report jocks, cheerleaders, and rich kids taking the situation into their own hands.  Members of the marching band, chorus, and various arts clubs remain at home.         


In [24]:
print("Data after cleansing : {}".format(blog_data['cleansed_text'][1]))

Data after cleansing : popular uprising in basra coalition forces report jocks cheerleaders and rich kids taking the situation into their own hands members of the marching band chorus and various arts clubs remain at home


Removing Stop words

In [25]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords=set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [26]:
blog_data['cleansed_text'] = blog_data['cleansed_text'].apply(lambda words: ' '.join(word for word in words.split() if word not in stopwords))

In [27]:
print("After removing stop words : {}".format(blog_data['cleansed_text'][1]))

After removing stop words : popular uprising basra coalition forces report jocks cheerleaders rich kids taking situation hands members marching band chorus various arts clubs remain home


3. As we want to make this into a multi-label classification problem, you are required to merge
all the label columns together, so that we have all the labels together for a particular sentence
(7.5 points)

a. Label columns to merge: “gender”, “age”, “topic”, “sign”

b. After completing the previous step, there should be only two columns in your data
frame i.e. “text” and “labels” as shown in the below image


In [28]:
blog_data['labels']=blog_data.apply(lambda col: [col['gender'],str(col['age']),col['topic'],col['sign']], axis=1)

In [29]:
blog_data=blog_data[['cleansed_text','labels']]

In [30]:
blog_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   cleansed_text  10000 non-null  object
 1   labels         10000 non-null  object
dtypes: object(2)
memory usage: 156.4+ KB


In [31]:
blog_data.head(10)

,cleansed_text,labels
0,good ole texas http www nytimes com opinion he...,"[male, 35, Technology, Aries]"
1,popular uprising basra coalition forces report...,"[male, 35, Technology, Aries]"
2,cannot believe supreme court used technicality...,"[male, 35, Technology, Aries]"
3,live tokyo years livetime dvd release updated ...,"[female, 24, indUnk, Aries]"
4,please note following using mobile receive pho...,"[male, 23, Internet, Aquarius]"
5,urllink resume form,"[male, 24, BusinessServices, Cancer]"
6,today veri fun lessons played round chess fren...,"[male, 14, Student, Libra]"
7,weeps laughs maniacally thrashes leather strap...,"[male, 35, Technology, Aries]"
8,hi anne sophie invited group however would lov...,"[female, 25, Student, Leo]"
9,watch napolean,"[male, 35, Technology, Aries]"


In [32]:
X=blog_data['cleansed_text']
y=blog_data['labels']

Separate features and labels, and split the data into training and testing (5 points)

In [33]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.30)

In [34]:
X_test.shape

(3000,)

Vectorize the features (5 points)

a. Create a Bag of Words using count vectorizer

  i. Use ngram_range=(1, 2)

  ii. Vectorize training and testing features

b. Print the term-document matrix

In [35]:
from sklearn.feature_extraction.text import CountVectorizer

In [36]:
vectorizer=CountVectorizer(binary=True, ngram_range=(1,2))

In [37]:
X_train=vectorizer.fit_transform(X_train)

In [38]:
X_train[0]

<1x468778 sparse matrix of type '<class 'numpy.int64'>'
	with 495 stored elements in Compressed Sparse Row format>

In [39]:
X_test=vectorizer.transform(X_test)

In [40]:
X_test[0]

<1x468778 sparse matrix of type '<class 'numpy.int64'>'
	with 24 stored elements in Compressed Sparse Row format>

In [41]:
vectorizer.get_feature_names()

['aa',
 'aa amazing',
 'aa keeps',
 'aa nice',
 'aaa',
 'aaa come',
 'aaa joe',
 'aaa looks',
 'aaa someone',
 'aaa take',
 'aaa tow',
 'aaa travel',
 'aaaa',
 'aaaa jet',
 'aaaaaaaaaaaah',
 'aaaaaaaaaaahhhhhhhhhhhhhhhhhhh',
 'aaaaaaaaaaahhhhhhhhhhhhhhhhhhh hw',
 'aaaaaaaaah',
 'aaaaaaah',
 'aaaaaaah fade',
 'aaaaack',
 'aaaaahhhh',
 'aaaaahhhh heath',
 'aaaah',
 'aaaah wisdom',
 'aaah',
 'aaah hafta',
 'aaahhh',
 'aaahhh cryptic',
 'aaahing',
 'aaahing much',
 'aaarrggghhh',
 'aaarrggghhh plus',
 'aaarrggghhh thanks',
 'aaarrrggghhhhhhhhgggghhhhhh',
 'aaarrrggghhhhhhhhgggghhhhhh dropped',
 'aaarrrrggggghhhhh',
 'aaarrrrggggghhhhh realized',
 'aactually',
 'aactually really',
 'aahed',
 'aahed god',
 'aahh',
 'aahh gw',
 'aal',
 'aal eliminate',
 'aal esseneth',
 'aal lost',
 'aal powerful',
 'aaldering',
 'aaldering urllink',
 'aamco',
 'aamco kind',
 'aand',
 'aand jim',
 'aar',
 'aar toy',
 'aarde',
 'aarde maak',
 'aargh',
 'aargh told',
 'aaron',
 'aaron aaron',
 'aaron already',


In [45]:
#This step is commented as runbook execution gets crashed for more number of input records(say >= 10000)
doc_matrix_df = pd.DataFrame(X_train.toarray(), columns = vectorizer.get_feature_names())

In [46]:
#This is created keeping input records as 1000 alone from the dataset for showing the output
doc_matrix_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Columns: 98395 entries, aa to zzzzz drop
dtypes: int64(98395)
memory usage: 525.5 MB


In [47]:
#This is created keeping input records as 1000 alone from the dataset for showing the output
doc_matrix_df.head(100)

,aa,aa anger,aaa,aaa travel,aaarrrggghhhhhhhhgggghhhhhh,aaarrrggghhhhhhhhgggghhhhhh dropped,aaldering,aaldering urllink,aand,aand jim,aarde,aarde maak,aaron,aaron rowand,aba,aba foundation,aba sessions,aba therapy,aba well,abandoned,abandoned although,abandons,abandons hate,abated,abated came,abbreviate,abbreviate fun,abc,abc last,abdomen,abdomen thighs,abdominal,abdominal area,aberration,aberration got,abiding,abiding pedestrians,abidjan,abidjan ivory,abilities,...,zknds smarterchild,zm,zm cos,zmattyb,zmattyb yahoo,zodiac,zodiac sign,zodiac similar,zombie,zombie right,zone,zone beyond,zone fuck,zone hope,zone large,zone one,zone somehow,zone work,zones,zones sometimes,zones today,zoniness,zoniness well,zoom,zoom first,zooms,zooms past,zovakware,zovakware lord,zua,zua watching,zun,zun charles,zun personally,zza,zza dong,zzzexy,zzzexy pathetic,zzzzz,zzzzz drop
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
96,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
97,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
98,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


6. Create a dictionary to get the count of every label i.e. the key will be label name and value will
be the total count of the label. Check below image for reference (5 points)

In [42]:
label_counts=dict()

for labels in blog_data.labels.values:
    for label in labels:
        if label in label_counts:
            label_counts[label]+=1
        else:
            label_counts[label]=1

In [43]:
label_counts

{'13': 42,
 '14': 212,
 '15': 602,
 '16': 440,
 '17': 1185,
 '23': 253,
 '24': 655,
 '25': 386,
 '26': 234,
 '27': 1054,
 '33': 136,
 '34': 553,
 '35': 2315,
 '36': 1708,
 '37': 33,
 '38': 46,
 '39': 79,
 '40': 1,
 '41': 20,
 '42': 14,
 '43': 6,
 '44': 3,
 '45': 16,
 '46': 7,
 'Accounting': 4,
 'Aquarius': 571,
 'Aries': 4198,
 'Arts': 45,
 'Automotive': 14,
 'Banking': 16,
 'BusinessServices': 91,
 'Cancer': 504,
 'Capricorn': 215,
 'Communications-Media': 99,
 'Consulting': 21,
 'Education': 270,
 'Engineering': 127,
 'Fashion': 1622,
 'Gemini': 150,
 'HumanResources': 2,
 'Internet': 118,
 'InvestmentBanking': 70,
 'Law': 11,
 'LawEnforcement-Security': 10,
 'Leo': 301,
 'Libra': 491,
 'Marketing': 156,
 'Museums-Libraries': 17,
 'Non-Profit': 71,
 'Pisces': 454,
 'Publishing': 4,
 'Religion': 9,
 'Sagittarius': 1097,
 'Science': 63,
 'Scorpio': 971,
 'Sports-Recreation': 80,
 'Student': 1137,
 'Taurus': 812,
 'Technology': 2654,
 'Telecommunications': 2,
 'Virgo': 236,
 'female': 4

7. Transform the labels - (7.5 points)
As we have noticed before, in this task each example can have multiple tags. To deal with
such kind of prediction, we need to transform labels in a binary form and the prediction will be
a mask of 0s and 1s. For this purpose, it is convenient to use MultiLabelBinarizer from sklearn

a. Convert your train and test labels using MultiLabelBinarizer


In [44]:
from sklearn.preprocessing import MultiLabelBinarizer
binarizer=MultiLabelBinarizer(classes=sorted(label_counts.keys()))

In [45]:
#y=binarizer.fit_transform(blog_data.labels)
y_train=binarizer.fit_transform(y_train)

In [46]:
y_test=binarizer.fit_transform(y_test)

8. Choose a classifier - (5 points)

In this task, we suggest using the One-vs-Rest approach, which is implemented in
OneVsRestClassifier class. In this approach k classifiers (= number of tags) are trained. As a basic classifier, use LogisticRegression. It is one of the simplest methods, but often it performs good enough in text classification tasks. It might take some time because the number of classifiers to train is large.

a. Use a linear classifier of your choice, wrap it up in OneVsRestClassifier to train it on every label

b. As One-vs-Rest approach might not have been discussed in the sessions, we are
providing you the code for that

In [47]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

In [48]:
model=LogisticRegression(solver='lbfgs',C=0.1,max_iter=150)
model=OneVsRestClassifier(model)
model.fit(X_train, y_train)

OneVsRestClassifier(estimator=LogisticRegression(C=0.1, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=150,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [49]:
y_pred=model.predict(X_test)

In [50]:
y_pred_inversed = binarizer.inverse_transform(y_pred)
y_test_inversed = binarizer.inverse_transform(y_test)

4. Display and explain detail the classification report

In [51]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score

print('Accuracy score: ', accuracy_score(y_test, y_pred))

print("Considering Average as Micro:")
print('F1 score: ', f1_score(y_test, y_pred, average='micro'))
print('Average precision score: ', average_precision_score(y_test, y_pred, average='micro'))
print('Average recall score: ', recall_score(y_test, y_pred, average='micro'))
print("")

print("Considering Average as Micro:")
print('F1 score: ', f1_score(y_test, y_pred, average='macro'))
print('Average precision score: ', average_precision_score(y_test, y_pred, average='macro'))
print('Average recall score: ', recall_score(y_test, y_pred, average='macro'))
print("")

print("Considering Average as Weighted:")
print('F1 score: ', f1_score(y_test, y_pred, average='weighted'))
print('Average precision score: ', average_precision_score(y_test, y_pred, average='weighted'))
print('Average recall score: ', recall_score(y_test, y_pred, average='weighted'))
print("")

print("Considering Average as Samples:")
print('F1 score: ', f1_score(y_test, y_pred, average='samples'))
print('Average precision score: ', average_precision_score(y_test, y_pred, average='samples'))
print('Average recall score: ', recall_score(y_test, y_pred, average='samples'))
print("")

Accuracy score:  0.265
Considering Average as Micro:
F1 score:  0.6088956253593268
Average precision score:  0.42856578797490535
Average recall score:  0.48541666666666666

Considering Average as Micro:
F1 score:  0.1764978984384521
Average precision score:  nan


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_ranking.py:677: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behav

Average recall score:  0.13370239643473328

Considering Average as Weighted:
F1 score:  0.5523721630405743
Average precision score:  0.4880298778488738
Average recall score:  0.48541666666666666

Considering Average as Samples:
F1 score:  0.5584063492063492


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Average precision score:  0.5010003472222221
Average recall score:  0.48541666666666666



Lets check the average parameter for f1-score:

average{‘micro’, ‘macro’, ‘samples’,’weighted’, ‘binary’} or None, default=’binary’
This parameter is required for multiclass/multilabel targets. If None, the scores for each class are returned. Otherwise, this determines the type of averaging performed on the data:

'binary':
Only report results for the class specified by pos_label. This is applicable only if targets (y_{true,pred}) are binary.

'micro':
Calculate metrics globally by counting the total true positives, false negatives and false positives.

'macro':
Calculate metrics for each label, and find their unweighted mean. This does not take label imbalance into account.

'weighted':
Calculate metrics for each label, and find their average weighted by support (the number of true instances for each label). This alters ‘macro’ to account for label imbalance; it can result in an F-score that is not between precision and recall.

'samples':
Calculate metrics for each instance, and find their average (only meaningful for multilabel classification where this differs from accuracy_score).

5. Print the true vs predicted labels for any 5 entries from the dataset.

In [73]:
import random
for i in range(1,5):
  r = random.randint(0, 1000)
  print("Predicted Label ", i, " ", "at index ", r, " ", y_pred_inversed[r])
  print("True Label ",i," ", "at index ", r, " ",y_test_inversed[r])

Predicted Label  1   at index  268   ('35', 'Aries', 'Technology', 'male')
True Label  1   at index  268   ('35', 'Aries', 'Technology', 'male')
Predicted Label  2   at index  200   ('Aries', 'male')
True Label  2   at index  200   ('36', 'Aries', 'Fashion', 'male')
Predicted Label  3   at index  867   ('Aries', 'male')
True Label  3   at index  867   ('35', 'Aries', 'Technology', 'male')
Predicted Label  4   at index  573   ('male',)
True Label  4   at index  573   ('33', 'Aquarius', 'InvestmentBanking', 'male')
